# Genome-Wide Association Study (GWAS) Workflow

## Table of Contents

1. [Introduction](#introduction)
2. [GWAS Quality Control](#QC)
4. [GWAS](#gwas)
5. [GWAS Results Output](#results)

<a id="introduction"></a>
## 1. Introduction

This notebook provides a comprehensive workflow on conducting a genome-wide association study (GWAS) using sample data. The workflow includes essential steps such as data inspection, quality control (QC), association testing, and visualization of results. The QC steps ensure that the data used in the analysis are of high quality, removing potential biases and errors.

#### Prerequisites
- **PLINK**: A widely used toolset for GWAS and population genetics, which can be downloaded from [here](https://www.cog-genomics.org/plink2).
- **R**: A programming language and environment for statistical computing and graphics, available [here](https://www.r-project.org).
- **'qqman' package in R**: For creating Manhattan and Q-Q plots for visualization of GWAS results.

### Workflow Overview
1. **Data Inspection**: Check the initial structure and content of the GWAS sample data.
2. **Conversion to Binary Files**: Convert the dataset to binary format for efficient processing.
3. **Missing Rate Calculation**: Identify and exclude SNPs and individuals with high missing rates.
4. **Heterozygosity Rate Calculation**: Detect and remove individuals with abnormal heterozygosity rates.
5. **Differential Missing Rate Test**: Exclude SNPs with significantly different missing rates between cases and controls.
6. **Hardy-Weinberg Equilibrium (HWE) Test**: Remove SNPs that deviate from HWE in controls.
7. **Linkage Disequilibrium (LD) Pruning**: Prune SNPs to remove those in high LD, reducing redundancy.
8. **Relatedness Check**: Identify and remove related individuals to ensure the independence of samples.
9. **Principal Component Analysis (PCA)**: Generate principal components to correct for population structure.
10. **GWAS**: Perform association testing to identify SNPs associated with the trait of interest.
11. **Visualization**: Create figures (e.g., Manhattan and Q-Q plots) to visualize the GWAS results.

Each step is demonstrated with corresponding commands and scripts to guide you through the entire process, from raw data to final results and visualizations. The goal is to provide a clear and practical understanding of conducting a GWAS with proper quality control and result interpretation. The user may need to make necessary adjustments with larger datasets.

### Environment Setup

In [1]:
# Ensure that the Project Root directory is set to 'prs_humans'
# Install and initialize renv if not already installed
if (!requireNamespace("renv", quietly = TRUE)) {
    install.packages("renv")
}

# Initialize renv and restore environment
# Note: You'll be told that there is one or more packages recorded in the lockfile that was not installed. 
# This is because lassosum will need to be 'manually' installed. I wrote a script to do this for you in the
# lassosum section.
setwd('..')
renv::init(bare = TRUE)
renv::restore()

- The library is already synchronized with the lockfile.


#### Load libraries

In [2]:
library(qqman)



For example usage please run: vignette('qqman')



Citation appreciated but not required:

Turner, (2018). qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. Journal of Open Source Software, 3(25), 731, https://doi.org/10.21105/joss.00731.





### Set the Project Root

In [3]:
# Verify the Project Root is set to prs_humans
library(here)
project_root <- here::here()

# Set the data directory
data_dir <- file.path(project_root, "data")
print(paste("Project root is set to:", project_root))
print(paste("data directory is set to:", data_dir))

here() starts at /Users/msoliai/Documents/git/gwas_workflow



[1] "Project root is set to: /Users/msoliai/Documents/git/gwas_workflow"
[1] "data directory is set to: /Users/msoliai/Documents/git/gwas_workflow/data"


In [4]:
# Set the seed
set.seed(123)

<a id="QC"></a>
## 2. GWAS Quality Control

### Step 1: Inspect data

In [5]:
# Execute the system command with the correct path
output <- system(paste("head", file.path(data_dir, "gwas_example.ped"), "| cut -c 1-100"), intern = TRUE)

# Print the output
if (length(output) > 0) {
  cat(output, sep = "\n")
} else {
  print("No output generated. Please check the file path and command.")
}

NA20505 NA20505 0 0 2 1 0 0 A A A C T T G G C T T T T T A A G G C C A A C C A A G G T T C C G G G G 
NA20504 NA20504 0 0 2 2 G G A A C C T T G G T T T T T T A A G G C C A A C C A A G G T T 0 0 G G G G 
NA20506 NA20506 0 0 2 2 G G A A A C T T G G T T T T C T A A A G C C C A C C A A A G T T C C G G G G 
NA20502 NA20502 0 0 2 1 G G A A C C T T G G T T T T C T A A A G C C C A C C A A A G T T T C G G G G 
NA20528 NA20528 0 0 1 2 G G A A C C G T G G C T G T C C G A A G C C C C C C A A A G T T C C G G G G 
NA20531 NA20531 0 0 2 1 G G A A A C G T G G C T T T C T G A G G C C C A T C A A G G T T C C G G G G 
NA20534 NA20534 0 0 1 1 C G G A C C G G G G C T G G C C G A A G C C C C C C A A G G T T C C G G A G 
NA20535 NA20535 0 0 2 1 G G A A A C G T G G C C T T C C G A A G C C C C C C C A A G T T C C G G G G 
NA20586 NA20586 0 0 1 2 G G A A A C G T G G T T G T C T A A A G C C C A T C C A A G T T C C G G G G 
NA20756 NA20756 0 0 2 1 G G A A A C G G G G C C T T C C G G G G C C A A C C A A G G C T T C

In [6]:
# Check if the file exists
file_path <- file.path(data_dir, "gwas_example.map")
if (file.exists(file_path)) {
  # Execute the system command with the correct path
  output <- system(paste("head", file_path), intern = TRUE)
  
  # Print the output
  if (length(output) > 0) {
    cat(output, sep = "\n")
  } else {
    print("No output generated from the command.")
  }
} else {
  print(paste("File not found:", file_path))
}

1	rs12565286	0	711153
1	rs12124819	0	766409
1	rs4970383	0	828418
1	rs13303118	0	908247
1	rs35940137	0	930066
1	rs2465136	0	980280
1	rs2488991	0	984254
1	rs3766192	0	1007060
1	rs10907177	0	1011209
1	rs9442398	0	1011558


### Step 2: Convert files to binary

In [7]:
# Convert to binary files
system(paste("plink --file", file.path(data_dir, "gwas_example"), "--make-bed --out", file.path(data_dir, "gwas_example")))

#### Verify that .bim and .fam files were written to the data directory

In [8]:
# List the files in the data directory
print(system(paste("ls -l", data_dir), intern = TRUE))

# Function to check file existence and run system command
run_system_command <- function(file_name) {
  file_path <- file.path(data_dir, file_name)
  if (file.exists(file_path)) {
    output <- system(paste("head", file_path), intern = TRUE)
    if (length(output) > 0) {
      cat(output, sep = "\n")
    } else {
      print(paste("No output generated from the command for", file_name))
    }
  } else {
    print(paste("File not found:", file_path))
  }
}

# Execute the system commands
run_system_command("gwas_example.bim")
run_system_command("gwas_example.fam")

[1] "total 176712"                                                        
[2] "-rw-r--r--  1 msoliai  staff   5015503 Jun  1 19:33 gwas_example.bed"
[3] "-rw-r--r--  1 msoliai  staff   2795442 Jun  1 19:33 gwas_example.bim"
[4] "-rw-r--r--  1 msoliai  staff      4472 Jun  1 19:33 gwas_example.fam"
[5] "-rw-r--r--  1 msoliai  staff      1481 Jun  1 19:33 gwas_example.log"
[6] "-rw-r--r--  1 msoliai  staff   2394202 Jun  1 10:08 gwas_example.map"
[7] "-rw-r--r--  1 msoliai  staff  80252472 Jun  1 10:08 gwas_example.ped"
1	rs12565286	0	711153	C	G
1	rs12124819	0	766409	G	A
1	rs4970383	0	828418	A	C
1	rs13303118	0	908247	G	T
1	rs35940137	0	930066	A	G
1	rs2465136	0	980280	C	T
1	rs2488991	0	984254	G	T
1	rs3766192	0	1007060	C	T
1	rs10907177	0	1011209	G	A
1	rs9442398	0	1011558	A	G
NA20505 NA20505 0 0 2 1
NA20504 NA20504 0 0 2 2
NA20506 NA20506 0 0 2 2
NA20502 NA20502 0 0 2 1
NA20528 NA20528 0 0 1 2
NA20531 NA20531 0 0 2 1
NA20534 NA20534 0 0 1 1
NA20535 NA20535 0 0 2 1
NA20586 NA20586 0 0 1 2
N

In [9]:
# List the details of the gwas_example.bed file
output_bed <- system(paste("ls -rlth", file.path(data_dir, "gwas_example.bed")), intern = TRUE)
if (length(output_bed) > 0) {
  cat(output_bed, sep = "\n")
} else {
  print("No output generated for gwas_example.bed")
}

# List the details of the dat/gwas_example.ped file
output_ped <- system(paste("ls -rlth", file.path(data_dir, "gwas_example.ped")), intern = TRUE)
if (length(output_ped) > 0) {
  cat(output_ped, sep = "\n")
} else {
  print("No output generated for gwas_example.ped")
}

-rw-r--r--  1 msoliai  staff   4.8M Jun  1 19:33 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.bed
-rw-r--r--  1 msoliai  staff    77M Jun  1 10:08 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.ped


### Step 3: Calculate the sample missingness rate

In [10]:
# Paths to the data files
bfile <- file.path(data_dir, "gwas_example")

# Calculate missing rate and capture output
plink_command <- paste("plink --bfile", bfile, "--missing --out", bfile)
output <- system(plink_command, intern = TRUE, ignore.stderr = TRUE)

# Print the output of the PLINK command
cat(output, sep = "\n")

# Check if the gwas_example.imiss file was created
imiss_file <- file.path(data_dir, "gwas_example.imiss")

if (file.exists(imiss_file)) {
  # Inspect the output missingness file
  output_imiss <- system(paste("head", imiss_file), intern = TRUE)
  if (length(output_imiss) > 0) {
    cat(output_imiss, sep = "\n")
  } else {
    print("No output generated for gwas_example.imiss")
  }

  # Use awk to filter individuals with missing rate greater than 2%
  output_awk <- system(paste("awk '$6 > 0.02'", imiss_file), intern = TRUE)
  if (length(output_awk) > 0) {
    cat(output_awk, sep = "\n")
  } else {
    print("No individuals with missing rate greater than 2% found.")
  }
} else {
  print(paste("File not found:", imiss_file))
}

PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.log.
Options in effect:
  --bfile /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example
  --missing
  --out /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example

16384 MB RAM detected; reserving 8192 MB for main workspace.
100310 variants loaded from .bim file.
200 people (101 males, 99 females) loaded from .fam.
200 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 200 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996374.
--missing: Sample mis

### Step 4: Calculate the heterozygosity rate

In [11]:
# Paths to the data files
bfile <- file.path(data_dir, "gwas_example")

# Calculate heterozygosity rate and capture output
plink_command <- paste("plink --bfile", bfile, "--het --out", bfile)
output <- system(plink_command, intern = TRUE, ignore.stderr = TRUE)

# Print the output of the PLINK command
cat(output, sep = "\n")

# Check if the gwas_example.het file was created
het_file <- file.path(data_dir, "gwas_example.het")

if (file.exists(het_file)) {
  # Inspect the output heterozygosity file
  output_het <- system(paste("head", het_file), intern = TRUE)
  if (length(output_het) > 0) {
    cat(output_het, sep = "\n")
  } else {
    print("No output generated for gwas_example.het")
  }
} else {
  print(paste("File not found:", het_file))
}

PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.log.
Options in effect:
  --bfile /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example
  --het
  --out /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example

16384 MB RAM detected; reserving 8192 MB for main workspace.
100310 variants loaded from .bim file.
200 people (101 males, 99 females) loaded from .fam.
200 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 200 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996374.
100310 variants and 200 p

### Step 5: Generate sample ID list failing missingness and heterozygosity checks

In [12]:
# Read the imiss and het files
imiss <- read.table(file.path(data_dir, "gwas_example.imiss"), header = TRUE)
het <- read.table(file.path(data_dir, "gwas_example.het"), header = TRUE)

# Calculate upper and lower limits for heterozygosity
upplimit <- mean(het$F) + (3 * sd(het$F))
lowlimit <- mean(het$F) - (3 * sd(het$F))

# Identify individuals to remove based on heterozygosity and missing rate
het.remove <- het[which(het$F < lowlimit | het$F > upplimit), c("FID", "IID")]
imiss.remove <- imiss[which(imiss$F_MISS > 0.02), c("FID", "IID")]

# Write the IDs to remove to a file in the data directory
output_file <- file.path(data_dir, "gwas_example.imiss-vs-het.remove")
write.table(rbind(het.remove, imiss.remove), output_file, append = FALSE, quote = FALSE, sep = " ", row.names = FALSE, col.names = FALSE)

print(paste("wrote IDs to remove to", output_file))

[1] "wrote IDs to remove to /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.imiss-vs-het.remove"


#### Remove samples/individuals in the list

In [13]:
# Remove samples listed in "gwas_example.imiss-vs-het.remove"
remove_file <- file.path(data_dir, "gwas_example.imiss-vs-het.remove")
output_file_qc1 <- file.path(data_dir, "gwas_example_qc1")

system(paste("plink --bfile", file.path(data_dir, "gwas_example"), "--remove", remove_file, "--make-bed --out", output_file_qc1))

### Step 7: Identify and remove variants with high missingness

This code focuses on overall SNP missingness across all samples. SNPs with more than 5% missing data are excluded.

In [14]:
# Inspect the missingness file
system(paste("head", file.path(data_dir, "gwas_example.lmiss")))

# gwas_example.lmiss.exclude contains the list of SNPs/variants that we will remove
system(paste("awk 'NR>1 && $5 > 0.05 {print $2}'", file.path(data_dir, "gwas_example.lmiss"), ">", file.path(data_dir, "gwas_example.lmiss.exclude")))

# Remove the specified SNPs/variants and create a new binary file
output_file_qc2 <- file.path(data_dir, "gwas_example_qc2")
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc1"), "--make-bed --exclude", file.path(data_dir, "gwas_example.lmiss.exclude"), "--out", output_file_qc2))

### Step 8: Remove variants with a high differential missingness rate

This code focuses on the differences in missingness rates between cases and controls. SNPs with significan differences > 5% are excluded to prevent potential bias in the association results.

In [15]:
# Test for differential missingness
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc2"), "--test-missing --out", file.path(data_dir, "gwas_example_qc2")))

# Inspect the missingness test output
system(paste("head", file.path(data_dir, "gwas_example_qc2.missing")))

# Identify SNPs with differential missingness greater than 5% and write their IDs to exclude file
system(paste("awk '$3-$4 > 0.05 || $3-$4 < -0.05 {print $2}'", file.path(data_dir, "gwas_example_qc2.missing"), ">", file.path(data_dir, "gwas_example_qc2.missing.exclude")))

# Remove the specified SNPs/variants and create a new binary file
output_file_qc3 <- file.path(data_dir, "gwas_example_qc3")
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc2"), "--exclude", file.path(data_dir, "gwas_example_qc2.missing.exclude"), "--make-bed --out", output_file_qc3))

### Step 9: Remove variants failing the HWE test

In [16]:
# Perform Hardy-Weinberg Equilibrium (HWE) test
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc3"), "--hardy --out", file.path(data_dir, "gwas_example_qc3")))

# Inspect the HWE test output
system(paste("head", file.path(data_dir, "gwas_example_qc3.hwe")))

# Identify SNPs that fail the HWE test in controls and write their IDs to exclude file
system(paste("awk '$3==\"UNAFF\" && $9 < 0.000001 {print $2}'", file.path(data_dir, "gwas_example_qc3.hwe"), ">", file.path(data_dir, "gwas_example_qc3.hwe.exclude")))

# Remove the specified SNPs/variants and create a new binary file
output_file_qc4 <- file.path(data_dir, "gwas_example_qc4")
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc3"), "--exclude", file.path(data_dir, "gwas_example_qc3.hwe.exclude"), "--make-bed --out", output_file_qc4))

### Step 10: LD pruning to remove related individuals

In [17]:
# Perform linkage disequilibrium (LD) pruning
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc4"), "--indep-pairwise 200 5 0.2 --out", file.path(data_dir, "gwas_example_qc4")))

# Inspect the LD pruning output
system(paste("head", file.path(data_dir, "gwas_example_qc4.prune.in")))

# Extract the pruned SNPs and create a new binary file
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc4"), "--extract", file.path(data_dir, "gwas_example_qc4.prune.in"), "--make-bed --out", file.path(data_dir, "gwas_example_qc4_indep")))

# Calculate genome-wide identity-by-descent (IBD)
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc4_indep"), "--genome --out", file.path(data_dir, "gwas_example_qc4_indep")))

# Inspect the genome-wide IBD output
system(paste("head", file.path(data_dir, "gwas_example_qc4_indep.genome")))

# Identify pairs of individuals with high relatedness (IBD > 0.2) and write their IDs to remove file
system(paste("awk 'NR>1 && $10 > 0.2 {print $1\"\\t\"$2}'", file.path(data_dir, "gwas_example_qc4_indep.genome"), ">", file.path(data_dir, "gwas_example_qc4_indep.genome.remove")))

# Remove the related individuals and create a final binary file
output_file_final <- file.path(data_dir, "gwas_example_final")
system(paste("plink --bfile", file.path(data_dir, "gwas_example_qc4"), "--remove", file.path(data_dir, "gwas_example_qc4_indep.genome.remove"), "--make-bed --out", output_file_final))

#### Verify the number of samples and variants that were removed

In [18]:
# Count the number of lines in the .bim and .fam files
output_bim <- system(paste("wc -l", file.path(data_dir, "gwas_example.bim")), intern = TRUE)
output_fam <- system(paste("wc -l", file.path(data_dir, "gwas_example.fam")), intern = TRUE)
cat(output_bim, sep = "\n")
cat(output_fam, sep = "\n")

# Count the number of lines in the final .bim and .fam files
output_final_bim <- system(paste("wc -l", file.path(data_dir, "gwas_example_final.bim")), intern = TRUE)
output_final_fam <- system(paste("wc -l", file.path(data_dir, "gwas_example_final.fam")), intern = TRUE)
cat(output_final_bim, sep = "\n")
cat(output_final_fam, sep = "\n")

  100310 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.bim
     200 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example.fam
   99712 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example_final.bim
     192 /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example_final.fam


### Step 11: Generate PC for population structure (using the new dataset without related individuals)

In [19]:
# Perform linkage disequilibrium (LD) pruning on the final dataset
ld_pruning_command <- paste("plink --bfile", file.path(data_dir, "gwas_example_final"), "--indep-pairwise 200 5 0.2 --out", file.path(data_dir, "gwas_example_final"))
ld_pruning_output <- system(ld_pruning_command, intern = TRUE, ignore.stderr = FALSE)
cat(ld_pruning_output, sep = "\n")

# Verify the existence of the pruning output file
prune_in_file <- file.path(data_dir, "gwas_example_final.prune.in")
if (file.exists(prune_in_file)) {
    cat("Pruning output file exists: ", prune_in_file, "\n")
} else {
    stop("Pruning output file does not exist: ", prune_in_file)
}

# Extract the pruned SNPs and create a new binary file
extract_command <- paste("plink --bfile", file.path(data_dir, "gwas_example_final"), "--extract", prune_in_file, "--make-bed --out", file.path(data_dir, "gwas_example_final_indep"))
extract_output <- system(extract_command, intern = TRUE, ignore.stderr = FALSE)
cat(extract_output, sep = "\n")

# Verify the existence of the new binary files
bed_file <- file.path(data_dir, "gwas_example_final_indep.bed")
bim_file <- file.path(data_dir, "gwas_example_final_indep.bim")
fam_file <- file.path(data_dir, "gwas_example_final_indep.fam")
if (file.exists(bed_file) && file.exists(bim_file) && file.exists(fam_file)) {
    cat("New binary files created successfully: \n")
    cat("BED file: ", bed_file, "\n")
    cat("BIM file: ", bim_file, "\n")
    cat("FAM file: ", fam_file, "\n")
} else {
    stop("One or more new binary files do not exist: ", bed_file, ", ", bim_file, ", ", fam_file)
}

# Perform PCA on the pruned dataset
pca_command <- paste("plink --bfile", file.path(data_dir, "gwas_example_final_indep"), "--pca --out", file.path(data_dir, "gwas_example_final_pca"))
pca_output <- system(pca_command, intern = TRUE, ignore.stderr = FALSE)
cat(pca_output, sep = "\n")

# Verify the existence of the PCA output file
pca_file <- file.path(data_dir, "gwas_example_final_pca.eigenvec")
if (file.exists(pca_file)) {
    cat("PCA output file exists: ", pca_file, "\n")
} else {
    stop("PCA output file does not exist: ", pca_file)
}

# Print the first few lines of the PCA results
pca_results <- read.table(pca_file, header = FALSE)
head(pca_results)

PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example_final.log.
Options in effect:
  --bfile /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example_final
  --indep-pairwise 200 5 0.2
  --out /Users/msoliai/Documents/git/gwas_workflow/data/gwas_example_final

16384 MB RAM detected; reserving 8192 MB for main workspace.
99712 variants loaded from .bim file.
192 people (95 males, 97 females) loaded from .fam.
192 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 192 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate i

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NA20505,NA20505,0.0757339,-0.0264009,0.0148166,-0.02375640,0.00887168,0.0032160,0.06784090,-0.0420065,⋯,-0.03899580,0.0670889,-0.01503630,0.0538179,0.04592370,-0.0145336,0.0825434,-0.0578123,0.01642010,-0.02207890
2,NA20504,NA20504,0.0688010,0.1941840,-0.0141943,-0.49767200,0.22632700,-0.0110324,-0.28426300,-0.0332691,⋯,-0.15103200,0.0929649,-0.00410145,-0.0557924,0.13054000,-0.0672311,-0.1084380,0.1498650,-0.00874232,0.03266830
3,NA20506,NA20506,0.0723643,-0.0178616,-0.0306206,0.01264690,-0.01306890,0.0553185,-0.03472080,0.0465207,⋯,-0.04249800,0.0662390,0.07750230,0.0505171,0.04081820,-0.0710421,-0.0148121,-0.0191663,-0.00394107,-0.00506505
4,NA20502,NA20502,0.0853343,0.1416060,0.0104528,-0.27112800,0.13741100,-0.0589519,0.03880530,-0.0619875,⋯,-0.03345590,0.0684865,0.12861600,0.0827991,-0.00887483,-0.0545222,0.4241820,0.0410108,-0.00196164,-0.01203590
5,NA20528,NA20528,0.0747823,-0.0190528,0.0129491,-0.01590730,0.04986240,-0.0344815,0.00888534,0.0657312,⋯,0.00186658,-0.0446660,0.01189170,0.0719046,0.05691540,-0.0721721,0.0315918,0.0778861,-0.08498170,-0.01238120
6,NA20531,NA20531,0.0813945,-0.0308249,0.0235611,0.00837153,0.01222940,0.0121661,0.06699310,-0.0780612,⋯,-0.13054000,-0.1030210,-0.03825250,0.1558770,0.03347760,-0.0280715,0.0160752,0.1545330,-0.08337830,-0.03708300


<a id="gwas"></a>
## 3. GWAS

### Perform the GWAS

In [20]:
# Perform GWAS with logistic regression, including the first 10 principal components as covariates
system(paste("plink --bfile", file.path(data_dir, "gwas_example_final"), 
             "--logistic hide-covar --covar", file.path(data_dir, "gwas_example_final_pca.eigenvec"), 
             "--covar-number 1-10 --out", file.path(data_dir, "gwas_example_final"), "--ci 0.95"))

#### Verify GWAS output

In [21]:
# Inspect the first few lines of the logistic regression association results
output_assoc <- system(paste("head", file.path(data_dir, "gwas_example_final.assoc.logistic")), intern = TRUE)
cat(output_assoc, sep = "\n")

 CHR                            SNP         BP   A1       TEST    NMISS         OR       SE      L95      U95         STAT            P 
   1                     rs12565286     711153    C        ADD      186     0.9459   0.6689   0.2549    3.509     -0.08317       0.9337
   1                      rs4970383     828418    A        ADD      192     0.7218   0.3336   0.3754    1.388      -0.9772       0.3285
   1                     rs13303118     908247    G        ADD      192     0.7256   0.2884   0.4123    1.277       -1.112       0.2661
   1                     rs35940137     930066    A        ADD      192      1.015   0.8313    0.199    5.179      0.01824       0.9855
   1                      rs2465136     980280    C        ADD      191     0.8569   0.3114   0.4655    1.577      -0.4961       0.6198
   1                      rs2488991     984254    G        ADD      192      0.934   0.3955   0.4302    2.028      -0.1727       0.8629
   1                      rs3766192    1007060 

<a id="results"></a>
## 4. GWAS Results Output

### Create a Manhattan plot

In [22]:
# Set the output directory
output_dir <- file.path(project_root, "output/figures")

# Read the logistic regression association results
assoc_adj <- read.table(file.path(project_root, "data/gwas_example_final.assoc.logistic"), header = TRUE)

# Filter out rows with missing P, OR, or STAT values
assoc_adj_final <- assoc_adj[which(!is.na(assoc_adj$P) & !is.na(assoc_adj$OR) & !is.na(assoc_adj$STAT)),]

# Identify significant results
assoc_adj_final_sig <- assoc_adj_final[assoc_adj_final[,12] < 0.00000005,]
print(assoc_adj_final_sig)

# Identify suggestive results
assoc_adj_final_sug <- assoc_adj_final[assoc_adj_final[,12] < 0.00001,]

# Load the qqman library
library(qqman)

# Generate Manhattan plot
pdf(file.path(output_dir, "assoc_adj_final.manhattan.pdf"))
manhattan(assoc_adj_final, main = "Manhattan Plot", ylim = c(0, 10))
dev.off()

# Generate Q-Q plot
pdf(file.path(output_dir, "assoc_adj_final.qq.pdf"))
qq(assoc_adj_final$P, main = "Q-Q Plot", ylim = c(0, 10))
dev.off()

# Calculate and print lambda GC
lambda_gc <- median((assoc_adj_final$STAT)^2) / qchisq(0.5, 1, lower.tail = FALSE)
print(paste("lambda GC:", lambda_gc))

 [1] CHR   SNP   BP    A1    TEST  NMISS OR    SE    L95   U95   STAT  P    
<0 rows> (or 0-length row.names)


pdf 
  2

pdf 
  2

[1] "lambda GC: 1.06051996604631"


In [23]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS Sonoma 14.5

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/Chicago
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] here_1.0.1  qqman_0.1.9

loaded via a namespace (and not attached):
 [1] digest_0.6.35     IRdisplay_1.1     utf8_1.2.4        base64enc_0.1-3  
 [5] fastmap_1.1.1     glue_1.7.0        htmltools_0.5.8.1 repr_1.1.7       
 [9] lifecycle_1.0.4   cli_3.6.2         fansi_1.0.6       vctrs_0.6.5      
[13] calibrate_1.7.7   renv_1.0.7        pbdZMQ_0.3-11     compiler_4.3.1   
[17] rprojroot_2.0.4   tools_4.3.